In [1]:
import os
from pathlib import Path
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import defaultdict

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from PIL import Image
from tqdm.auto import tqdm

import torchvision.transforms as T
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights


c:\Users\Nolan\Documents\My dokuments\GoIT\Deep_learning\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
IS_KAGGLE = False

if IS_KAGGLE:
    COMP_PATH = Path("../input/deep-learning-for-computer-vision-and-nlp-2026-01")
    FOLDS_PATH = Path("../input/petfinder-train-folds/train_folds.csv")  # зміниться під твою назву на Kaggle
else:
    COMP_PATH = Path("../data/raw")
    FOLDS_PATH = Path("../data/artifacts/train_folds.csv")

TRAIN_CSV = COMP_PATH / "train.csv"
TEST_CSV  = COMP_PATH / "test.csv"
IMG_DIR_TRAIN   = COMP_PATH / "images/images/train"
IMG_DIR_TEST    = COMP_PATH / "images/images/test"
IMG_DIR = COMP_PATH / "images/images"

print("train exists:", TRAIN_CSV.exists())
print("test exists :", TEST_CSV.exists())
print("images dir  :", IMG_DIR.exists())
print("folds exists:", FOLDS_PATH.exists())


train exists: True
test exists : True
images dir  : True
folds exists: True


In [3]:
train_df = pd.read_csv(TRAIN_CSV)
test_df  = pd.read_csv(TEST_CSV)
folds_df = pd.read_csv(FOLDS_PATH)

print(train_df.shape, test_df.shape, folds_df.shape)
train_df.head()


(6431, 3) (1891, 2) (6431, 3)


,PetID,Description,AdoptionSpeed
0,d3b4f29f8,Mayleen and Flo are two lovely adorable sister...,2
1,e9dc82251,A total of 5 beautiful Tabbys available for ad...,2
2,8111f6d4a,Two-and-a-half month old girl. Very manja and ...,2
3,693a90fda,Neil is a healthy and active ~2-month-old fema...,2
4,9d08c85ef,Gray kitten available for adoption in sungai p...,2


In [4]:
train_df = train_df.merge(folds_df, on=["PetID", "AdoptionSpeed"], how="inner")
print("after merge:", train_df.shape)
train_df[["PetID","AdoptionSpeed","fold"]].head()


after merge: (6431, 4)


,PetID,AdoptionSpeed,fold
0,d3b4f29f8,2,3
1,e9dc82251,2,3
2,8111f6d4a,2,2
3,693a90fda,2,4
4,9d08c85ef,2,3


In [5]:
train_imgs = list(IMG_DIR_TRAIN.glob("*"))
print("total image files:", len(train_imgs))
print("sample:", [p.name for p in train_imgs[:10]])

total image files: 28472
sample: ['000a290e4-1.jpg', '000a290e4-2.jpg', '000fb9572-1.jpg', '000fb9572-2.jpg', '000fb9572-3.jpg', '000fb9572-4.jpg', '000fb9572-5.jpg', '000fb9572-6.jpg', '001b1507c-1.jpg', '001b1507c-2.jpg']


In [6]:
# Check matching between image files and CSV PetIDs

petids_in_csv = set(train_df["PetID"].astype(str))

sample_files = random.sample(train_imgs, 200)
prefixes = [p.name.split("-", 1)[0] for p in sample_files]

match_rate = sum(pref in petids_in_csv for pref in prefixes) / len(prefixes)

print("Match rate:", match_rate)
print("Example prefixes:", prefixes[:10])


Match rate: 1.0
Example prefixes: ['7196383a7', '37fadf608', '97a80f8b3', '43c661026', '32ac38a9f', '6b240dd0b', '9e1caf559', '85ca390a0', '1a5e5d34c', 'eb6bb82fd']


In [7]:
petid_to_images = defaultdict(list)

for img_path in train_imgs:
    pet_id = img_path.name.split("-", 1)[0]
    petid_to_images[pet_id].append(img_path)

print("Total PetIDs with images:", len(petid_to_images))

Total PetIDs with images: 6431


In [18]:
test_imgs = list(IMG_DIR_TEST.glob("*"))
print("total image files:", len(test_imgs))
print("sample:", [p.name for p in test_imgs[:10]])

total image files: 9448
sample: ['0008c5398-1.jpg', '0008c5398-2.jpg', '0008c5398-3.jpg', '0008c5398-4.jpg', '0008c5398-5.jpg', '0008c5398-6.jpg', '004c2f355-1.jpg', '004c2f355-2.jpg', '00553ae55-1.jpg', '00553ae55-2.jpg']


In [21]:
# Check matching between image files and CSV PetIDs

petids_in_csv = set(test_df["PetID"].astype(str))

sample_files = random.sample(test_imgs, 200)
prefixes = [p.name.split("-", 1)[0] for p in sample_files]

match_rate = sum(pref in petids_in_csv for pref in prefixes) / len(prefixes)

print("Match rate:", match_rate)
print("Example prefixes:", prefixes[:10])

Match rate: 0.99
Example prefixes: ['d8fc9bc28', '61be9a64f', '7d52177d6', 'd320b7121', 'c7b3c09d4', '982dce813', 'bbfc5274f', '29466dcf4', '1d5a59ca9', 'a0909c2bb']


In [19]:
petid_to_images_test = defaultdict(list)

for img_path in test_imgs:
    pet_id = img_path.name.split("-", 1)[0]
    petid_to_images_test[pet_id].append(img_path)

print("Total PetIDs with images:", len(petid_to_images_test))
print("Total PetIDs with images:", len(petid_to_images))

Total PetIDs with images: 1899
Total PetIDs with images: 6431


In [22]:
available_test_petids = set(petid_to_images_test.keys())

before = len(test_df)
test_df = test_df[test_df["PetID"].isin(available_test_petids)].reset_index(drop=True)
after = len(test_df)

print(f"Filtered test_df: {before} -> {after}")

Filtered test_df: 1891 -> 1887


In [8]:
# example output
some_pet = list(petid_to_images.keys())[3]
print(some_pet)
print(petid_to_images[some_pet])


002230dea
[WindowsPath('../data/raw/images/images/train/002230dea-1.jpg'), WindowsPath('../data/raw/images/images/train/002230dea-2.jpg'), WindowsPath('../data/raw/images/images/train/002230dea-3.jpg'), WindowsPath('../data/raw/images/images/train/002230dea-4.jpg')]


In [9]:
class PetDataset(Dataset):
    def __init__(self, df, petid_to_images, transform=None):
        self.df = df.reset_index(drop=True)
        self.petid_to_images = petid_to_images
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        pet_id = row["PetID"]
        label = row["AdoptionSpeed"] - 1  # make labels 0-based

        img_path = random.choice(self.petid_to_images[pet_id])  # baseline: randomly choose one image
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label


In [ ]:
n_rows, n_cols = 4, 3
n = n_rows * n_cols

sample_paths = random.sample(train_imgs, k=n)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 16))
axes = axes.flatten()

for ax, p in zip(axes, sample_paths):
    img = Image.open(p).convert("RGB")
    ax.imshow(img)
    ax.set_title(p.name, fontsize=9)
    ax.axis("off")

plt.tight_layout()
plt.show()

In [11]:
IMG_SIZE = 224

weights = EfficientNet_B0_Weights.DEFAULT
mean = weights.transforms().mean
std  = weights.transforms().std

train_tfms = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.RandomHorizontalFlip(p=0.5),
    T.ToTensor(),
    T.Normalize(mean=mean, std=std),
])

val_tfms = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.ToTensor(),
    T.Normalize(mean=mean, std=std),
])

In [12]:
FOLD = 0

df_tr = train_df[train_df["fold"] != FOLD].reset_index(drop=True)
df_va = train_df[train_df["fold"] == FOLD].reset_index(drop=True)

train_ds = PetDataset(df_tr, petid_to_images, transform=train_tfms)
val_ds   = PetDataset(df_va, petid_to_images, transform=val_tfms)

len(train_ds), len(val_ds)

(5144, 1287)

In [14]:
BATCH_SIZE = 32
NUM_WORKERS = 0  

train_loader = DataLoader(
    train_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=False,     
    drop_last=True       
)

val_loader = DataLoader(
    val_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=False,
    drop_last=False
)

print("train batches:", len(train_loader))
print("val batches  :", len(val_loader))


train batches: 160
val batches  : 41


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

device: cpu


In [16]:
NUM_CLASSES = 4

def build_model(num_classes=NUM_CLASSES):
    weights = EfficientNet_B0_Weights.DEFAULT
    model = efficientnet_b0(weights=weights)

    in_features = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(in_features, num_classes)
    return model

In [17]:
# Створюємо модель
model = build_model()

# Перевіряємо архітектуру
print(model.classifier) 
# Має вивести щось типу:
# Sequential(
#   (0): Dropout(p=0.2, inplace=True)
#   (1): Linear(in_features=1280, out_features=4, bias=True)  <-- ТУТ МАЄ БУТИ 4!
# )

# Перевіряємо на фейкових даних
dummy_input = torch.randn(2, 3, 224, 224) # 2 картинки, 3 канали, 224x224
output = model(dummy_input)
print("Output shape:", output.shape) 
# Має бути: torch.Size([2, 4]) -> (2 картинки, 4 класи)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to C:\Users\Nolan/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [01:04<00:00, 334kB/s]


Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=4, bias=True)
)
Output shape: torch.Size([2, 4])


In [ ]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()  # режим навчання (вмикає dropout, batchnorm у train-режимі)

    running_loss = 0.0
    correct = 0
    total = 0

    pbar = tqdm(loader, desc="train", leave=False)

    for x, y in pbar:
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()          # обнуляємо градієнти
        logits = model(x)              # прямий прохід
        loss = criterion(logits, y)    # рахуємо loss

        loss.backward()                # зворотний прохід (градієнти)
        optimizer.step()               # крок оптимізатора

        bs = x.size(0)
        running_loss += loss.item() * bs
        total += bs

        preds = logits.argmax(dim=1)
        correct += (preds == y).sum().item()

        pbar.set_postfix(loss=float(loss.item()))

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

In [ ]:
@torch.no_grad()
def valid_epoch_with_probs(model, loader, criterion, device):
    model.eval()  # режим оцінки (вимикає dropout тощо)

    running_loss = 0.0
    correct = 0
    total = 0

    probs_list = []

    pbar = tqdm(loader, desc="valid", leave=False)

    for x, y in pbar:
        x = x.to(device)
        y = y.to(device)

        logits = model(x)
        loss = criterion(logits, y)

        bs = x.size(0)
        running_loss += loss.item() * bs
        total += bs

        preds = logits.argmax(dim=1)
        correct += (preds == y).sum().item()

        probs = F.softmax(logits, dim=1).detach().cpu().numpy()  # (bs, 4)
        probs_list.append(probs)

        pbar.set_postfix(loss=float(loss.item()))

    val_loss = running_loss / total
    val_acc = correct / total
    val_probs = np.vstack(probs_list)  # (N_val, 4)

    return val_loss, val_acc, val_probs


In [23]:
missing_train = sum(pid not in petid_to_images for pid in train_df["PetID"].values)
missing_test  = sum(pid not in petid_to_images_test for pid in test_df["PetID"].values)

print("missing train petids:", missing_train)
print("missing test  petids:", missing_test)


missing train petids: 0
missing test  petids: 0


In [ ]:
N_SPLITS = 5
EPOCHS = 3
LR = 1e-3

BATCH_SIZE = 32
NUM_WORKERS = 2     
PIN_MEMORY = True


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

criterion = nn.CrossEntropyLoss()

# OOF probabilities для кожного рядка train_df (N, 4)
oof_probs = np.zeros((len(train_df), NUM_CLASSES), dtype=np.float32)


# 5-FOLD LOOP

for fold in range(N_SPLITS):
    print(f"\n===== FOLD {fold} / {N_SPLITS - 1} =====")

    df_tr = train_df[train_df["fold"] != fold].reset_index(drop=True)
    df_va = train_df[train_df["fold"] == fold].reset_index(drop=True)

    train_ds = PetDataset(df_tr, petid_to_images, transform=train_tfms)
    val_ds   = PetDataset(df_va, petid_to_images, transform=val_tfms)

    train_loader = DataLoader(
        train_ds,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        drop_last=True
    )

    val_loader = DataLoader(
        val_ds,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        drop_last=False
    )

    model = build_model(NUM_CLASSES).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)

    # ---- train epochs ----
    for epoch in range(EPOCHS):
        tr_loss, tr_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
        va_loss, va_acc, _ = valid_epoch_with_probs(model, val_loader, criterion, device)
        print(f"epoch {epoch+1}/{EPOCHS} | train loss {tr_loss:.4f} acc {tr_acc:.4f} | val loss {va_loss:.4f} acc {va_acc:.4f}")

    # ---- final val probs (OOF) ----
    va_loss, va_acc, fold_probs = valid_epoch_with_probs(model, val_loader, criterion, device)
    print(f"FOLD {fold} FINAL | val loss {va_loss:.4f} acc {va_acc:.4f}")

    # запис у правильні рядки train_df
    val_index = train_df.index[train_df["fold"] == fold].to_numpy()
    oof_probs[val_index] = fold_probs


In [ ]:
oof_out = pd.DataFrame({
    "PetID": train_df["PetID"].values,
    "fold": train_df["fold"].values,
})
for c in range(NUM_CLASSES):
    oof_out[f"pred_{c}"] = oof_probs[:, c]

oof_out.to_csv("cnn_oof.csv", index=False)
print("Saved cnn_oof.csv")


In [ ]:
DO_TEST = True   # якщо False — блок тесту просто не виконується

class PetTestDataset(Dataset):
    def __init__(self, df, petid_to_images_test, transform=None):
        self.df = df.reset_index(drop=True)
        self.petid_to_images = petid_to_images_test
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        pet_id = row["PetID"]

        img_path = random.choice(self.petid_to_images[pet_id])
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, pet_id


In [ ]:
@torch.no_grad()
def predict_test_probs(model, loader, device):
    model.eval()
    probs_list = []
    petids = []

    pbar = tqdm(loader, desc="test", leave=False)
    for x, pid in pbar:
        x = x.to(device)
        logits = model(x)
        probs = F.softmax(logits, dim=1).detach().cpu().numpy()

        probs_list.append(probs)
        petids.extend(list(pid))

    return np.array(petids), np.vstack(probs_list)


In [ ]:
if DO_TEST:
    test_probs_accum = np.zeros((len(test_df), NUM_CLASSES), dtype=np.float32)

In [ ]:
    # ---- TEST PROBS ----
if DO_TEST:
    test_ds = PetTestDataset(test_df, petid_to_images_test, transform=val_tfms)
    test_loader = DataLoader(
        test_ds,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        drop_last=False
    )

    _, fold_test_probs = predict_test_probs(model, test_loader, device)
    test_probs_accum += fold_test_probs / N_SPLITS


In [ ]:
if DO_TEST:
    test_out = pd.DataFrame({"PetID": test_df["PetID"].values})
    for c in range(NUM_CLASSES):
        test_out[f"pred_{c}"] = test_probs_accum[:, c]

    test_out.to_csv("cnn_test.csv", index=False)
    print("Saved cnn_test.csv")
